In [1]:
import os
import json
import random
import requests
from openai import OpenAI
from nemo_microservices import NeMoMicroservices

In [2]:
os.environ["WANDB_API_KEY"]="e0dae5c27937567b0d7c792082d063b2d7f7eed6"

In [3]:
WANDB_API_KEY = os.getenv("WANDB_API_KEY")

In [4]:
from config import *

# Initialize NeMo Microservices SDK client
nemo_client = NeMoMicroservices(
    base_url=NEMO_URL,
    inference_base_url=NIM_URL,
)

In [10]:
nemo_client.projects.list()

SyncDefaultPagination[Project](object='list', data=[], sort='created_at', pagination=DefaultPaginationPagination(current_page_size=0, page=1, page_size=10, total_pages=0, total_results=0))

In [11]:
print(f"Data Store endpoint: {NDS_URL}")
print(f"Entity Store, Customizer, Evaluator endpoint: {NEMO_URL}")
print(f"NIM endpoint: {NIM_URL}")
print(f"Namespace: {NMS_NAMESPACE}")
print(f"Base Model for Customization: {BASE_MODEL}@{BASE_MODEL_VERSION}")

Data Store endpoint: http://data-store.test
Entity Store, Customizer, Evaluator endpoint: http://nemo.test
NIM endpoint: http://nim.test
Namespace: lora-tutorial-ns
Base Model for Customization: meta/llama-3.2-1b-instruct@v1.0.0+A100


In [17]:
# Path where data preparation notebook saved finetuning and evaluation data
DATA_ROOT = os.path.join(os.getcwd(), "data")
# CUSTOMIZATION_DATA_ROOT = os.path.join(DATA_ROOT, "customization")
# VALIDATION_DATA_ROOT = os.path.join(DATA_ROOT, "validation")
# EVALUATION_DATA_ROOT = os.path.join(DATA_ROOT, "evaluation")

# Sanity checks
train_fp = f"{DATA_ROOT}/training.jsonl"
assert os.path.exists(train_fp), f"The training data at '{train_fp}' does not exist. Please ensure that the data was prepared successfully."

val_fp = f"{DATA_ROOT}/validation.jsonl"
assert os.path.exists(val_fp), f"The validation data at '{val_fp}' does not exist. Please ensure that the data was prepared successfully."

test_fp = f"{DATA_ROOT}/test.jsonl"
assert os.path.exists(test_fp), f"The test data at '{test_fp}' does not exist. Please ensure that the data was prepared successfully."

In [18]:
def create_namespaces(nemo_client, ds_host, namespace):
    # Create namespace in Entity Store
    try:
        namespace_obj = nemo_client.namespaces.create(id=namespace)
        print(f"Created namespace in Entity Store: {namespace_obj.id}")
    except Exception as e:
        # Handle if namespace already exists
        if "409" in str(e) or "422" in str(e):
            print(f"Namespace {namespace} already exists in Entity Store")
        else:
            raise e

    # Create namespace in Data Store (still using requests as SDK doesn't cover Data Store)
    nds_url = f"{ds_host}/v1/datastore/namespaces"
    resp = requests.post(nds_url, data={"namespace": namespace})
    assert resp.status_code in (200, 201, 409, 422), \
        f"Unexpected response from Data Store during namespace creation: {resp.status_code}"
    print(f"Data Store namespace creation response: {resp}")

In [19]:
create_namespaces(nemo_client=nemo_client, ds_host=NDS_URL, namespace=NMS_NAMESPACE)

Namespace lora-tutorial-ns already exists in Entity Store
Data Store namespace creation response: <Response [409]>


In [20]:
 # Verify Namespace in Data Store (using requests as SDK doesn't cover Data Store)
response = requests.get(f"{NDS_URL}/v1/datastore/namespaces/{NMS_NAMESPACE}")
print(f"Data Store - Status Code: {response.status_code}\nResponse JSON: {response.json()}")

# Verify Namespace in Entity Store
namespace_obj = nemo_client.namespaces.retrieve(namespace_id=NMS_NAMESPACE)
print(f"\nEntity Store - Namespace: {namespace_obj.id}")
print(f"Created at: {namespace_obj.created_at}")
print(f"Description: {namespace_obj.description}")
print(f"Project: {namespace_obj.project}")

Data Store - Status Code: 201
Response JSON: {'namespace': 'lora-tutorial-ns', 'created_at': '2025-08-13T14:28:28Z', 'updated_at': '2025-08-13T14:33:28Z'}

Entity Store - Namespace: lora-tutorial-ns
Created at: 2025-08-13 14:28:28.058716
Description: None
Project: None


In [21]:
namespace_obj

Namespace(id='lora-tutorial-ns', created_at=datetime.datetime(2025, 8, 13, 14, 28, 28, 58716), custom_fields={}, description=None, ownership=None, project=None, updated_at=datetime.datetime(2025, 8, 13, 14, 28, 28, 58719))

In [22]:
repo_id = f"{NMS_NAMESPACE}/{DATASET_NAME}" 

In [23]:
from huggingface_hub import HfApi

hf_api = HfApi(endpoint=f"{NDS_URL}/v1/hf", token="")

# Create repo
hf_api.create_repo(
    repo_id=repo_id,
    repo_type='dataset',
)

/home/ubuntu/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HfHubHTTPError: 409 Client Error: Conflict for url: http://data-store.test/v1/hf/api/repos/create

You already created this repo

In [24]:
hf_api.upload_file(path_or_fileobj=train_fp,
    path_in_repo="training/train.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

hf_api.upload_file(path_or_fileobj=val_fp,
    path_in_repo="validation/val.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

hf_api.upload_file(path_or_fileobj=test_fp,
    path_in_repo="testing/test.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

CommitInfo(commit_url='', commit_message='Upload testing/test.jsonl with huggingface_hub', commit_description='', oid='a94c9b8bcf70efc7cf80c008a3c033450d2a246a', pr_url=None, repo_url=RepoUrl('', endpoint='https://huggingface.co', repo_type='model', repo_id=''), pr_revision=None, pr_num=None)

In [ ]:
# nemo_client.datasets.delete(dataset_name=DATASET_NAME, namespace=NMS_NAMESPACE)

DeleteResponse(id='dataset-2ibXH5f1wRC1jzn1ws9sQu', deleted_at=None, message='Resource deleted successfully.')

In [28]:
 # Create dataset
dataset = nemo_client.datasets.create(
    name=DATASET_NAME,
    namespace=NMS_NAMESPACE,
    description="News Dataset for FSI Blueprint",
    files_url=f"hf://datasets/{NMS_NAMESPACE}/{DATASET_NAME}",
    project="tool_calling",
    
)
print(f"Created dataset: {dataset.namespace}/{dataset.name}")
dataset

Created dataset: lora-tutorial-ns/news-lora-dataset


Dataset(files_url='hf://datasets/lora-tutorial-ns/news-lora-dataset', id='dataset-MvfFbbDRxmLWS8nmc7WGMX', created_at=datetime.datetime(2025, 8, 13, 16, 41, 25, 865354), custom_fields={}, description='News Dataset for FSI Blueprint', format=None, hf_endpoint=None, limit=None, name='news-lora-dataset', namespace='lora-tutorial-ns', project='tool_calling', split=None, updated_at=datetime.datetime(2025, 8, 13, 16, 41, 25, 865357))

In [12]:
# Sanity check to validate dataset
dataset_obj = nemo_client.datasets.retrieve(namespace=NMS_NAMESPACE, dataset_name=DATASET_NAME)

print("Files URL:", dataset_obj.files_url)
assert dataset_obj.files_url == f"hf://datasets/{repo_id}"

Files URL: hf://datasets/lora-tutorial-ns/news-lora-dataset


In [36]:
f"{BASE_MODEL}@{BASE_MODEL_VERSION}"

'meta/llama-3.2-1b-instruct@v1.0.0+A100'

In [45]:
# List customization configs with filters
configs = nemo_client.customization.configs.list(
    page=1,
    page_size=10,
    sort="-created_at",
    filter={
        "training_type": "sft",
        "finetuning_type": "lora",
        "enabled": True
    }
)

print(f"Found {len(configs.data)} configs")
for config in configs.data:
    print(f"Namespace: {config.namespace} Config:{config.name} - {config.description}")

Found 2 configs
Namespace: meta Config:llama-3.2-1b-instruct@v1.0.0+L40 - None
Namespace: meta Config:llama-3.2-1b-instruct@v1.0.0+A100 - None


In [135]:
CUSTOM_MODEL = 'lora-tutorial-ns/llama-3.2-1b-xlam-run1@v5'

In [ ]:
# nemo_client.models.delete(namespace= NMS_NAMESPACE,model_name='llama-3.2-1b-xlam-run1@v2')

DeleteResponse(id='model-YUsHWp4sVSgpEBKzmW64JJ', deleted_at=None, message='Resource deleted successfully.')

In [136]:
# Create customization job
# If WANDB_API_KEY is set, we send it in the request header, which will report the training metrics to Weights & Biases (WandB).
if WANDB_API_KEY:
    client_with_wandb = nemo_client.with_options(default_headers={"wandb-api-key": WANDB_API_KEY})
else:
    client_with_wandb = nemo_client

customization = client_with_wandb.customization.jobs.create(
    name="llama-3.2-1b-xlam-ft-seq-packed",
    output_model=CUSTOM_MODEL,
    config=f"{BASE_MODEL}@{BASE_MODEL_VERSION}",
    dataset={"name": DATASET_NAME, "namespace": NMS_NAMESPACE},
    
    hyperparameters={
        "sequence_packing_enabled": True,
        "training_type": "sft",
        "finetuning_type": "lora",
        "epochs": 2,
        "batch_size": 16,
        "learning_rate": 0.0001,
        "lora": {
            "adapter_dim": 16,
            "adapter_dropout": 0.1
        }
    }
)
print(f"Created customization job: {customization.id}")
customization

Created customization job: cust-VJ7FRPk9FLuH6RyMLYDyMb


CustomizationJob(config='meta/llama-3.2-1b-instruct@v1.0.0+A100', dataset='lora-tutorial-ns/news-lora-dataset', hyperparameters=Hyperparameters(finetuning_type='lora', batch_size=16, distillation=None, dpo=None, epochs=2, learning_rate=0.0001, log_every_n_steps=None, lora=LoraParameters(adapter_dim=16, adapter_dropout=0.1, alpha=16, target_modules=None), sequence_packing_enabled=True, sft=None, training_type='sft', val_check_interval=None, weight_decay=None), id='cust-VJ7FRPk9FLuH6RyMLYDyMb', config_snapshot=CustomizationConfigJobValue(base_model='meta/llama-3.2-1b-instruct', max_seq_length=4096, precision='bf16-mixed', training_option=CustomizationTrainingOption(finetuning_type='lora', micro_batch_size=1, num_gpus=1, training_type='sft', data_parallel_size=1, num_nodes=1, pipeline_parallel_size=1, tensor_parallel_size=1, use_sequence_parallel=False), dataset_schema=None, prompt_template='{prompt} {completion}'), created_at=datetime.datetime(2025, 8, 13, 18, 17, 19, 29071), dataset_par

In [137]:
# To track status
JOB_ID = customization.id

customization = nemo_client.customization.jobs.retrieve(JOB_ID)

# This will be the name of the model that will be used to send inference queries to
CUSTOMIZED_MODEL = customization.output_model

In [74]:
# nemo_client.customization.jobs.cancel(job_id=JOB_ID)

In [138]:
 # Get job status
job_status = nemo_client.customization.jobs.status(job_id=JOB_ID)

print("Percentage done:", job_status.percentage_done)
print("Job Status:", json.dumps(job_status.model_dump(), indent=2, default=str))

Percentage done: 0.0
Job Status: {
  "created_at": "2025-08-13 18:17:19.419953",
  "status": "created",
  "updated_at": "2025-08-13 18:17:19.419953",
  "best_epoch": null,
  "elapsed_time": 0.0,
  "epochs_completed": 0,
  "metrics": null,
  "percentage_done": 0.0,
  "status_logs": [
    {
      "updated_at": "2025-08-13 18:17:19.419953",
      "detail": null,
      "message": "created"
    }
  ],
  "steps_completed": 0,
  "steps_per_epoch": null,
  "train_loss": null,
  "val_loss": null
}


In [ ]:
 # Add wait job function to wait for the customization job to complete

from time import sleep, time

def wait_job(nemo_client, job_id: str, polling_interval: int = 10, timeout: int = 6000):
    """Helper for waiting an eval job using SDK."""
    start_time = time()
    job = nemo_client.customization.jobs.retrieve(job_id=job_id)
    status = job.status

    while (status in ["pending", "created", "running"]):
        # Check for timeout
        if time() - start_time > timeout:
            raise RuntimeError(f"Took more than {timeout} seconds.")

        # Sleep before polling again
        sleep(polling_interval)

        # Fetch updated status and progress
        job = nemo_client.customization.jobs.retrieve(job_id=job_id)
        status = job.status
        progress = 0.0
        if status == "running" and job.status_details:
            progress = job.status_details.percentage_done or 0.0
        elif status == "completed":
            progress = 100

        print(f"Job status: {status} after {time() - start_time:.2f} seconds. Progress: {progress}%")


    return job

job = wait_job(nemo_client, JOB_ID, polling_interval=5, timeout=2400)

# Wait for 2 minutes, because sometimes, the job is finished, but the finetuned model is not ready in NIM yet.
sleep(120)

Job status: running after 5.25 seconds. Progress: 50.0%
Job status: running after 10.32 seconds. Progress: 50.0%
Job status: running after 15.39 seconds. Progress: 50.0%
Job status: running after 20.56 seconds. Progress: 50.0%
Job status: running after 25.63 seconds. Progress: 50.0%
Job status: running after 30.70 seconds. Progress: 50.0%
Job status: running after 35.76 seconds. Progress: 50.0%
Job status: running after 40.83 seconds. Progress: 50.0%
Job status: running after 45.90 seconds. Progress: 50.0%
Job status: running after 50.96 seconds. Progress: 50.0%
Job status: running after 56.03 seconds. Progress: 50.0%
Job status: running after 61.10 seconds. Progress: 50.0%
Job status: running after 66.17 seconds. Progress: 50.0%
Job status: running after 71.24 seconds. Progress: 50.0%
Job status: running after 76.31 seconds. Progress: 50.0%
Job status: running after 81.48 seconds. Progress: 50.0%
Job status: running after 86.54 seconds. Progress: 50.0%
Job status: running after 91.61 

In [77]:
 # List models with filters
models_page = nemo_client.models.list(
    filter={"namespace": NMS_NAMESPACE},
    sort="-created_at"
)

# Print models information
print(f"Found {len(models_page.data)} models in namespace {NMS_NAMESPACE}:")
for model in models_page.data:
    print(f"\nModel: {model.name}")
    print(f"  Namespace: {model.namespace}")
    print(f"  Base Model: {model.base_model}")
    print(f"  Created: {model.created_at}")
    if model.peft:
        print(f"  Fine-tuning Type: {model.peft.finetuning_type}")

Found 2 models in namespace lora-tutorial-ns:

Model: llama-3.2-1b-xlam-run1@v4
  Namespace: lora-tutorial-ns
  Base Model: meta/llama-3.2-1b-instruct
  Created: 2025-08-13 17:02:36.340413
  Fine-tuning Type: lora

Model: llama-3.2-1b-xlam-run1@v1
  Namespace: lora-tutorial-ns
  Base Model: meta/llama-3.2-1b-instruct
  Created: 2025-08-13 14:33:29.640105
  Fine-tuning Type: lora


In [78]:
# CUSTOMIZED_MODEL is constructed as `namespace/model_name`, so we need to extract the model name
model = nemo_client.models.retrieve(namespace=NMS_NAMESPACE, model_name=CUSTOMIZED_MODEL.split("/")[1])

print(f"Model: {model.namespace}/{model.name}")
print(f"Base Model: {model.base_model}")
print(f"Status: {model.artifact.status}")

Model: lora-tutorial-ns/llama-3.2-1b-xlam-run1@v4
Base Model: meta/llama-3.2-1b-instruct
Status: upload_completed


In [79]:
 # Check if the custom LoRA model is hosted by NVIDIA NIM
models = nemo_client.inference.models.list()
model_names = [model.id for model in models.data]

assert CUSTOMIZED_MODEL in model_names, \
    f"Model {CUSTOMIZED_MODEL} not found" 

In [80]:
def read_jsonl(file_path):
    """Reads a JSON Lines file and yields parsed JSON objects"""
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if not line:
                continue  # Skip empty lines
            try:
                yield json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue


test_data = list(read_jsonl(test_fp))

print(f"There are {len(test_data)} examples in the test set")

There are 7676 examples in the test set


In [108]:
def create_message_batches(data_list, batch_size=100):
    """
    Creates batches of messages from a list of dictionaries.
    
    Args:
        data_list: List of dictionaries containing 'prompt' key
        batch_size: Size of each batch (default: 100)
    
    Returns:
        List of batches, where each batch contains message dictionaries
    """
    batches = []
    
    for i in range(0, len(data_list), batch_size):
        batch = data_list[i:i + batch_size]
        
        # Create messages for this batch
        batch_messages = []
        for test_sample in batch:
            messages = {"role": "user", "content": test_sample['prompt']}
            batch_messages.append(messages)
        
        batches.append(batch_messages)
    
    return batches

# Usage example:
message_batches = create_message_batches(test_data, batch_size=100)

print(f"Created {len(message_batches)} batches")
print(f"First batch has {len(message_batches[0])} messages")
print(f"Sample message from first batch: {message_batches[0][0]}")

Created 77 batches
First batch has 100 messages
Sample message from first batch: {'role': 'user', 'content': "Given the following headline:\n### START HEADLINE ###\n\nHearing Endo Int'l Held Talks to Sell Paladin Labs to Knight Therapeutics\n\n### END HEADLINE ###\n\nWhat event type best classifies it? Choose from the following list:\n\n-analyst rating\n-price targets\n-earnings\n-labour related\n-mergers and acquisitions\n-dividends\n-regulatory\n-stock price movement\n-credit ratings\n-products-services\n-product approval\n-guidance\n-other\n\nProvide only the event type putting it inside double square brackets and in a new line like:\n[[label]]\n\n### START EVENT OUTPUT ###\n\n"}


In [ ]:
responses = []
c=0
for batch in message_batches:
    for message in batch:
        completion = nemo_client.chat.completions.create(model = CUSTOMIZED_MODEL,
                                            messages = [message],
                                            temperature = 0.1,
                                            top_p = 0.7,
                                            max_tokens = 512,
                                            stream = False
                                            )
        resp = completion.choices[0].message.content
        responses.append(resp)
        c += 1
        if c % 500 == 0:
            print(f"Processed {c} messages")

Processed 100 messages
Processed 200 messages
Processed 300 messages
Processed 400 messages
Processed 500 messages
Processed 600 messages
Processed 700 messages
Processed 800 messages
Processed 900 messages
Processed 1000 messages
Processed 1100 messages
Processed 1200 messages
Processed 1300 messages
Processed 1400 messages
Processed 1500 messages
Processed 1600 messages
Processed 1700 messages
Processed 1800 messages
Processed 1900 messages
Processed 2000 messages
Processed 2100 messages
Processed 2200 messages
Processed 2300 messages
Processed 2400 messages
Processed 2500 messages
Processed 2600 messages
Processed 2700 messages
Processed 2800 messages
Processed 2900 messages
Processed 3000 messages
Processed 3100 messages
Processed 3200 messages
Processed 3300 messages
Processed 3400 messages
Processed 3500 messages
Processed 3600 messages
Processed 3700 messages
Processed 3800 messages
Processed 3900 messages
Processed 4000 messages
Processed 4100 messages
Processed 4200 messages
P

In [125]:
# Clean responses using lambda to remove square brackets
responses = list(map(lambda x: x.replace('[', '').replace(']', ''), responses))

In [127]:
test_data[0]

{'prompt': "Given the following headline:\n### START HEADLINE ###\n\nHearing Endo Int'l Held Talks to Sell Paladin Labs to Knight Therapeutics\n\n### END HEADLINE ###\n\nWhat event type best classifies it? Choose from the following list:\n\n-analyst rating\n-price targets\n-earnings\n-labour related\n-mergers and acquisitions\n-dividends\n-regulatory\n-stock price movement\n-credit ratings\n-products-services\n-product approval\n-guidance\n-other\n\nProvide only the event type putting it inside double square brackets and in a new line like:\n[[label]]\n\n### START EVENT OUTPUT ###\n\n",
 'completion': 'mergers and acquisitions'}

In [128]:
true_labels = [i['completion'] for i in test_data]

In [133]:
set(true_labels)

{'analyst rating',
 'credit ratings',
 'dividends',
 'earnings',
 'guidance',
 'labour issues',
 'mergers and acquisitions',
 'no event',
 'other',
 'price targets',
 'product approval',
 'products-services',
 'regulatory',
 'stock price movement'}

In [134]:
set(responses)

{'-earnings',
 'Dividends',
 'Earnings',
 'Fitch Affirming',
 'Fitch Affirms',
 'Fitch Affirms Ratings on AmEx, Discover, SLM; Outlook Stables',
 'Fitch Downgrades',
 'Fitch Expects',
 'Hearing',
 'Hearing Jana',
 'IPO',
 'IPO Outlook For The Week: Boats, Fertility Solutions, Risk Management And Chinese Supplements',
 'IPO Wrapup for the Week of April 7th, 2014',
 'Mergers and acquisitions',
 'Stock price movement',
 'UPDATE',
 'analysis',
 'analysis rating',
 'analyst rating',
 'analyzer rating',
 'credit ratings',
 'dividends',
 'earnings',
 'event',
 'event rating',
 'event type',
 'eventing',
 'events-services',
 'filing',
 'guidance',
 'guide',
 'hearing',
 'issue notification',
 'issues',
 'joint application development and marketing',
 'joint venture',
 'judge rating',
 'jury awarding',
 'jury verdict',
 'label',
 'labor issues',
 'labour issues',
 'license agreement',
 'mergers and acquisitions',
 "news corporation in hot water over hacking of murdered teen's phone",
 'policy',

In [131]:
from sklearn.metrics import classification_report

In [132]:
classification_report(y_true=true_labels,
    y_pred=responses,
    zero_division=0,
    output_dict=True
)

{'-earnings': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Dividends': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Earnings': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Fitch Affirming': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Fitch Affirms': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Fitch Affirms Ratings on AmEx, Discover, SLM; Outlook Stables': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Fitch Downgrades': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Fitch Expects': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'Hearing': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0},
 'Hearing Jana': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 0.0},
 'IPO': {'precision': 0.0, 'recall': 0.0, 'f1-score':

In [ ]:
print(f"Name of your custom model is: {CUSTOMIZED_MODEL}") 

'[[label]]'